<font style="font-size:40px">Example lesson</font>
<p><font style="font-size:26px">Bootstrapping for better sales reports</font></p>

---

1. [Introduction](#h1_introduction)
1. [Workspace setup](#h1_workspace_setup)
1. [Load data](#h1_load_data)
1. [Data exploration](#h1_data_exploration)
1. [Data cleaning](#h1_data_cleaning)
1. [Create a basic sales report](#h1_basic_sales_report)
1. [Create a better sales report](#h1_better_sales_report)

# Introduction
<a id='h1_introduction'></a>

In this session, we will create a basic sales report for a data set from an online retailer. We will then apply bootstrapping to identify which of the apparent trends are statistically significant.

## Learning goals
* Use pandas `groupby()`, `agg()` and `apply()` for dataframe aggregation.
* Use pandas `apply()` with a custom aggregate function to produce a basic sales report.
* Modify our custom aggregate function to produce a bootstrapped sales report.

## Assumed prior knowledge
Python:
* __pandas__: Dataframe interrogation and slicing. Familiarity with `groupby()`, `agg()` and `apply()` beneficial but not necessary.
* __numpy__: `np.random.choice()`.
* List comprehension.

Other:
* Some familiarity with the concepts of bootstrapping and Monte Carlo methods. (Perhaps these were introduced in the previous session.)

# Workspace setup
<a id='h1_workspace_setup'></a>

In [ ]:
# Run a few common commands.
%run workspace_setup.py

## Imports

In [ ]:
# Common.
import os
import numpy as np
import pandas as pd

# Charting.
import seaborn as sns

# Misc.
from IPython.display import Image # Display an image.

In [ ]:
# `qa_fns` contains a few functions for producing charts and dataframes for this lesson.
import functions.qa_fns as qa

In [ ]:
#%aimport functions.qa_fns

# Load data
<a id='h1_load_data'></a>
`retail_df` is a pandas dataframe containing sales data for an online retailer.

The dataframe has been saved in the pickle format instead of the more common CSV. The main advantage of the pickle format is that it faithfully preserves the dataframe object, including column dtypes.

Data source: https://www.kaggle.com/mashlyn/online-retail-ii-uci.

In [ ]:
retail_df = pd.read_pickle(os.path.join('..', 'data', 'retail.pkl'))
retail_df.shape

## Key

Column name | Description
--- | ---
Index: `invoice_id` | Unique order ID. Prefix "C" indicates a cancelled order.
`invoice_date_dt` | Datetime of order.
`period_id` | Year and quarter of order.
`country` | Country order placed from.
`order_value` | Order value in GBP.

# Data exploration
<a id='h1_data_exploration'></a>

In [ ]:
# A reminder of why data exploration is important.
# https://genomebiology.biomedcentral.com/articles/10.1186/s13059-020-02133-w.
Image(os.path.join('..', 'images', 'bmi_steps.png'), retina=True)

## Quick glance

Spend a few minutes examining the dataframe, `retail_df`, any way you like.

Some functions that might be useful:
* `pd.DataFrame.head(n)`
* `pd.DataFrame.sample(n)`
* `pd.DataFrame.describe()`
* `pd.DataFrame.dtypes()`
* `set()` or `pd.Series.unique()`
* `pd.Series.sum()`; `pd.Series.mean()`
* `pd.DataFrame.groupby(...).describe()`
* `pd.DataFrame.groupby(...).agg(...)`

Example: `retail_df.sample(10)`.

In [ ]:
retail_df.head()

In [ ]:
retail_df.sample(5)

In [ ]:
len(retail_df['country'].unique())

In [ ]:
retail_df.describe()

In [ ]:
retail_df.dtypes

In [ ]:
retail_df['period_id'].dtype

Do any of the histograms look "unusual"?

## Aggregation
We will use the pandas dataframe methods `groupby()`, `agg()` and `sort_values()` to compute some aggregate statistics on `retail_df`.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html

An example of the syntax is given below.

In [ ]:
n_invoices_by_country_df = retail_df.groupby([
    'country'
]).agg(
    n_invoices=('order_value', 'count') # Syntax: new_aggregate_column_name=('existing_column_name', 'builtin_function_name' or custom_function_name).
).sort_values(
    by='n_invoices',
    ascending=False
)

n_invoices_by_country_df.head(10)

### Plot

In [ ]:
n_invoices_by_country_df.plot.bar(
    log=True,
    figsize=(15, 6)
);

## Exercise
Modify the code that generated `n_invoices_by_country_df` to create a new dataframe, `sales_by_country_df`. You will need to use the aggregate function "sum" instead of "count".

Call `plot.bar()` on this new dataframe, as we did above.

In [ ]:
sales_by_country_df = retail_df.groupby([
    'country'
]).agg(
    sales=('order_value', 'sum'),
).sort_values(
    by='sales',
    ascending=False
)

sales_by_country_df.plot.bar(
    log=True,
    figsize=(15, 6)
);

## Exercise
The UK seems to have much higher sales than the other countries.

What fraction of all sales value comes from the UK?

In [ ]:
sum(retail_df.loc[retail_df['country'] == 'United Kingdom', 'order_value']) / sum(retail_df['order_value'])

## Filter `retail_df` to just the countries with highest sales
There are a lot of countries with very low sales. Let's focus on just the top 12 countries for the remainder of the session.

In [ ]:
top_selling_countries = sales_by_country_df.index[:12]
top_selling_countries

In [ ]:
len(retail_df)

In [ ]:
retail_df = retail_df.loc[retail_df['country'].isin(top_selling_countries)]

In [ ]:
len(retail_df)

## Histograms
We will plot a histogram of order value for each country.

There is a pandas built-in method, `hist()`, for plotting these histograms with a single command. However, the results are not always visually clear.

In [ ]:
retail_df['order_value'].hist(
    by=retail_df['country'],
    figsize=(15, 10),
    #bins=40
);

We can generate better-looking histograms by employing seaborn.

A wrapper function, `seaborn_histplot_grid()`, is provided for expedience.

In [ ]:
qa.seaborn_histplot_grid(retail_df, value_colname='order_value', groupby_colname='country', groupby_categories=top_selling_countries)

# Data cleaning
<a id='h1_data_cleaning'></a>

## Remove invoices with value £0.00

In [ ]:
len(retail_df.loc[retail_df['order_value'] == 0])

In [ ]:
len(retail_df)

In [ ]:
retail_df = retail_df.loc[retail_df['order_value'] != 0]

In [ ]:
len(retail_df)

In [ ]:
qa.seaborn_histplot_grid(retail_df, value_colname='order_value', groupby_colname='country', groupby_categories=top_selling_countries)

# Create a basic sales report
<a id='h1_basic_sales_report'></a>

<font style="font-size:20px">Goal</font> \
Compare the total sales in period __Q1_2010__ with the total sales in period __Q1_2011__, country by country.

The output should be a dataframe with `country` as the index, and the following columns:
* The number of orders in {Q1_2010, Q1_2011}.
* The total sales in {Q1_2010, Q1_2011}.
* The change in sales (increase or decrease) in £ from Q1_2010 to Q1_2011.
* The relative change in sales, expressed as a fraction. E.g. 0.25 means a 25% increase in sales; -0.05 means a 5% decrease.

In [ ]:
period_id_0, period_id_1 = 'Q1_2010', 'Q1_2011'

## Create a custom aggregate function for use with `apply`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.apply.html

### Example function

In [ ]:
def example_apply_fn(df: pd.DataFrame) -> pd.Series:
    '''Simple example function for use as follows: `pd.DataFrame.groupby(...).apply(example_apply_fn)`.
    To be compatible with `apply()`, a function must take a pandas DataFrame as its first argument and return a DataFrame, Series or scalar.'''
    return pd.Series(
        {
            'sales':      df['order_value'].sum(),
            'n_invoices': len(df['order_value'])
        }
    )

In [ ]:
retail_df.groupby([   # The `groupby` column(s) become the index of the output dataframe.
    'country'
]).apply(             # The function specified in `apply` is applied to each of the dataframes returned by `groupby`.
    example_apply_fn  # The outputs of this function are stitched together to form the output dataframe.
)

### Write a `compare_sales()` function

Now we will write a new function for use with `apply`. The function must create an output dataframe as specified in the Goal.

A reminder of the required columns:
* The number of orders in {Q1_2010, Q1_2011}.
* The total sales in {Q1_2010, Q1_2011}.
* The change in sales (increase or decrease) in £ from Q1_2010 to Q1_2011.
* The relative change in sales, expressed as a fraction. E.g. 0.25 means a 25% increase in sales; -0.05 means a 5% decrease.

Let's also allow the names of the two periods being compared to be passed as arguments.

In [ ]:
def compare_sales(df: pd.DataFrame, period_id_0: str, period_id_1: str) -> pd.Series:
    '''Usage: `retail_df.groupby('country').apply(compare_sales, period_id_0, period_id_1)`.
    
    Args:
        period_id_0 (str): Name of the first period to compare--matching one of the periods from the `period_id` column of `retail_df`. E.g. 'Q1_2011'.
        period_id_1 (str): Name of the second period to compare. E.g. 'Q1_2010'.
    '''
    df_p0 = df.loc[df['period_id'] == period_id_0]
    df_p1 = df.loc[df['period_id'] == period_id_1]
    
    return pd.Series(
        {
            f'n_orders_{period_id_0}':  len(df_p0),
            f'n_orders_{period_id_1}':  len(df_p1),
            f'sales_{period_id_0}':     ...,
            f'sales_{period_id_1}':     ...,
            'Δ_sales':                  ... - ...,
            'rel_change_sales':         ... / ... - 1
        }
    )

In [ ]:
def compare_sales(df: pd.DataFrame, period_id_0: str, period_id_1: str) -> pd.Series:
    '''Usage: `retail_df.groupby('country').apply(compare_sales, period_id_0, period_id_1)`.
    
    Args:
        period_id_0 (str): Name of the first period to compare--matching one of the periods from the `period_id` column of `retail_df`. E.g. 'Q1_2011'.
        period_id_1 (str): Name of the second period to compare. E.g. 'Q1_2010'.
    '''
    df_p0 = df.loc[df['period_id'] == period_id_0]
    df_p1 = df.loc[df['period_id'] == period_id_1]
    
    p0_sales_sum = df_p0['order_value'].sum()
    p1_sales_sum = df_p1['order_value'].sum()
    
    return pd.Series(
        {
            f'n_orders_{period_id_0}':  len(df_p0),
            f'n_orders_{period_id_1}':  len(df_p1),
            f'sales_{period_id_0}':     p0_sales_sum,
            f'sales_{period_id_1}':     p1_sales_sum,
            'Δ_sales':                  p1_sales_sum - p0_sales_sum,
            'rel_change_sales':         p1_sales_sum / p0_sales_sum - 1
        }
    )

### Test your function

In [ ]:
# sales_by_country = retail_df.loc[
#     retail_df['period_id'].isin([period_id_0, period_id_1])

sales_by_country = retail_df.groupby([
    'country'
]).apply(                     # Note the syntax: arguments to our `compare_sales()` function must be passed as keyword arguments (`kwargs`) to `apply()`.
    compare_sales,
    period_id_0=period_id_0,
    period_id_1=period_id_1
).reindex(                    # An optional step: order the output dataframe by `top_selling_countries`.
    top_selling_countries
)

sales_by_country

## Improve the readability of the sales dataframe
The function `sales_dataframe_styler()` is provided to improve the readability of the sales summary dataframes generated during this lesson.

In [ ]:
qa.sales_dataframe_styler(sales_by_country, period_id_0, period_id_1)

Our basic sales report is now complete.

Questions:
* Does this table tell the whole story?
* What important information is missing (if any)?
* How can we improve this sales report?

# Create a better sales report
<a id='h1_better_sales_report'></a>

> “It is easy to lie with statistics. It is hard to tell the truth without statistics.”
>
> — Andrejs Dunkels

The basic sales report dataframe provides some standard metrics of sales performance. However, these figures lack any measure of statistical significance.

It would be much better if the sales report gave us a __confidence interval__ around each of the figures of interest. These confidence intervals can be estimated via __bootstrapping__.

## Bootstrapping: a quick reminder
Bootstrapping is a method of sampling with replacement from a data set in order to generate a new, unbiased sample from approximately the same distribution that generated the original data.

In [ ]:
for i in range(3):
    print(np.random.choice(range(10), size=10, replace=True))

In [ ]:
def compare_sales_bootstrapped(df: pd.DataFrame, period_id_0: str, period_id_1: str, n_boot: int=5000) -> pd.Series:
    '''An upgraded version of `compare_sales` which adds bootstrapped sales metrics. Bootstrapped columns have the prefix "bs_".'''
    df_p0 = df.loc[df['period_id'] == period_id_0]
    df_p1 = df.loc[df['period_id'] == period_id_1]
    
    p1_minus_p0 = []
    p1_div_p0 = []
    
    # Bootstrap loop.
    for i in range(n_boot):
        # Draw a bootstrap sample of order value for Period 0 and take the sum. Do the same for Period 1.
        bs_p0_sales = np.sum(np.random.choice(df_p0['order_value'], size=len(df_p0), replace=True))
        bs_p1_sales = np.sum(np.random.choice(df_p1['order_value'], size=len(df_p1), replace=True))
        
        # Append the difference of the bootstrapped sales values to list `p1_minus_p0`, and their quotient to `p1_div_p0`.
        p1_minus_p0.append(bs_p1_sales - bs_p0_sales)
        p1_div_p0.append(bs_p1_sales / bs_p0_sales)
    
    # Compute the unbootstrapped (i.e. actual) sales for each Period.
    p0_sales = df_p0['order_value'].sum()
    p1_sales = df_p1['order_value'].sum()
    
    # Compute the percentiles for the bootstrapped £ change and relative change in sales between Period 0 and Period 1.
    Δ_percentiles = np.quantile(p1_minus_p0, [0.025, 0.5, 0.975])
    rel_change_percentiles = np.quantile(p1_div_p0, [0.025, 0.5, 0.975])

    return pd.Series(
        {
            f'n_orders_{period_id_0}':     len(df_p0),                       # Number of orders for Period 0.
            f'n_orders_{period_id_1}':     len(df_p1),                       # Number of orders for Period 1.
            f'sales_{period_id_0}':        p0_sales,                         # Unbootstrapped (i.e. actual) sales for Period 0.
            f'sales_{period_id_1}':        p1_sales,                         # Unbootstrapped (i.e. actual) sales for Period 1.
            'Δ_sales':                     p1_sales - p0_sales,              # £ difference/change in sales between Period 0 and Period 1.
            'rel_change_sales':            p1_sales / p0_sales - 1,          # Relative change in sales between Period 0 and Period 1. E.g. 0.25 means a +25% change.
            'bs_Δ_sales_mean':             np.nanmean(p1_minus_p0),          # Mean of bootstraps of £ change in sales between Period 0 and Period 1.
            'bs_Δ_sales_sd':               np.nanstd(p1_minus_p0),           # Standard deviation of bootstraps of £ change in sales between Period 0 and Period 1.
            'bs_Δ_sales_q0.025':           Δ_percentiles[0],                 # 2.5th percentile of bootstraps of £ change in sales between Period 0 and Period 1.
            'bs_Δ_sales_q0.5':             Δ_percentiles[1],                 # 50th percentile (i.e. median) of bootstraps of £ change in sales between Period 0 and Period 1.
            'bs_Δ_sales_q0.975':           Δ_percentiles[2],                 # 97.5th percentile of bootstraps of £ change in sales between Period 0 and Period 1.
            'bs_rel_change_sales_mean':    np.nanmean(p1_div_p0),            # Mean of bootstraps of relative change in sales between Period 0 and Period 1.
            'bs_rel_change_sales_sd':      np.nanstd(p1_div_p0),             # Standard deviation of bootstraps of relative change in sales between Period 0 and Period 1.
            'bs_rel_change_sales_q0.025':  rel_change_percentiles[0] - 1,    # 2.5th percentile of bootstraps of relative change in sales between Period 0 and Period 1.
            'bs_rel_change_sales_q0.5':    rel_change_percentiles[1] - 1,    # 50th percentile of bootstraps of relative change in sales between Period 0 and Period 1.
            'bs_rel_change_sales_q0.975':  rel_change_percentiles[2] - 1,    # 97.5th percentile of bootstraps of relative change in sales between Period 0 and Period 1.
            f'{period_id_1}_>_{period_id_0}_frac': np.mean(                  # The fraction of bootstraps in which the sales in Period 1 exceeded those in Period 0.
                [quotient >= 1 for quotient in p1_div_p0]
            )
        }
    )

In [ ]:
%%time
# ~ 16 s for n_boot == 20,000.
bs_sales_by_country = retail_df.groupby(
    ['country']
).apply(
    compare_sales_bootstrapped,
    period_id_0=period_id_0,
    period_id_1=period_id_1,
    n_boot=20000
).reindex(
    top_selling_countries
)

In [ ]:
qa.sales_dataframe_styler(bs_sales_by_country, period_id_0, period_id_1)

In [ ]:
qa.sales_dataframe_styler(
    bs_sales_by_country[[colname for colname in bs_sales_by_country.columns if ('Δ' in colname) or ('n_orders' in colname)] + [bs_sales_by_country.columns[-1]]],
    period_id_0,
    period_id_1
)

In [ ]:
qa.sales_dataframe_styler(
    bs_sales_by_country[[colname for colname in bs_sales_by_country.columns if ('rel_change' in colname) or ('n_orders' in colname)] + [bs_sales_by_country.columns[-1]]].drop(columns=['bs_rel_change_sales_mean', 'bs_rel_change_sales_sd']),
    period_id_0,
    period_id_1
)

> “Offered the choice between mastery of a five-foot shelf of analytical statistics books and middling ability at performing statistical Monte Carlo simulations, we would surely choose to have the latter skill.”
>
> _— Numerical Recipes: The Art of Scientific Computing_